# Customers and Products Analysis
The goal of this project is to analyze data from a sales records database for scale model cars and extract information for decision-making. We will be extracting key performance indicators(KPIs) to make smart decisions for sales. Sales data analysis will cover many aspects - sales, production, customer experience, employee efficiency.

We will find answers to the following questions:
- Question 1: Which products should we order more of or less of?
- Question 2: How should we tailor marketing and communication strategies to customer behaviors?
- Question 3: How much can we spend on acquiring new customers?

[Database File](stores.db)


![Fig: Database Schema](stores_db_schema.png)

Database contains eight tables:

1. Customers: customer data
2. Employees: all employee information
3. Offices: sales office information
4. Orders: customers' sales orders
5. OrderDetails: sales order line for each sales order
6. Payments: customers' payment records
Products: a list of scale model cars
ProductLines: a list of product line categories

#### In order to understand the relationship between the tables, a schema diagram showing the foreign key lines is below

![](stores_db_schema_with_fk.png)

In [19]:
# Enable SQL magic
%load_ext sql

In [20]:
#Establish a connection to the database
%sql sqlite:///stores.db

In [55]:
%%sql
-- Get table name, number of rows and number of columns
SELECT 'orders' AS table_name,
       (SELECT COUNT(*) FROM orders) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('orders')) AS column_count
       
UNION ALL

SELECT 'customers' AS table_name,
       (SELECT COUNT(*) FROM customers) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('customers')) AS column_count

UNION ALL

SELECT 'offices' AS table_name,
       (SELECT COUNT(*) FROM offices) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('offices')) AS column_count

UNION ALL

SELECT 'orderdetails' AS table_name,
       (SELECT COUNT(*) FROM orderdetails) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('orderdetails')) AS column_count

UNION ALL

SELECT 'payments' AS table_name,
       (SELECT COUNT(*) FROM payments) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('payments')) AS column_count

UNION ALL

SELECT 'productlines' AS table_name,
       (SELECT COUNT(*) FROM productlines) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('productlines')) AS column_count

UNION ALL

SELECT 'products' AS table_name,
       (SELECT COUNT(*) FROM products) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('products')) AS column_count

UNION ALL

SELECT 'employees' AS table_name,
       (SELECT COUNT(*) FROM employees) AS row_count,
       (SELECT COUNT(*) FROM PRAGMA_TABLE_INFO('employees')) AS column_count



 * sqlite:///stores.db
Done.


table_name,row_count,column_count
orders,326,7
customers,122,13
offices,7,9
orderdetails,2996,5
payments,273,4
productlines,7,4
products,110,9
employees,23,8


In [50]:
#import sqlite3 to use python to fetch data
import sqlite3
import pandas as pd
# Create a connection object to the database
conn = sqlite3.connect('stores.db')



### Question 1: Which Products Should We Order More of or Less of?

This question refers to inventory reports, including low stock(i.e. product in demand) and product performance. This will optimize the supply and the user experience by preventing the best-selling products from going out-of-stock

- The low stock represents the quantity of the sum of each product ordered divided by the quantity of product in stock. We can consider the ten highest rates. These will be the top ten products that are almost out-of-stock or completely out-of-stock.
- The product performance represents the sum of sales per product.
- Priority products for restocking are those with high product performance that are on the brink of being out of stock.

In order to perform the following calculations:  
    $low stock = SUM(quantityOrdered)/quantityInStock$  
    $product performance = SUM(quantityOrdered * priceEach)$  

We will need following tables:
![](products-orders-tables.png)


Group the oderdetails table by productCode to add up quantityOrdered for each product.
Then join this table with products table to get the quantityInStock column. Divide the two to get the low_stock indicator. Order in descending order and limit the result to top 10 low stock products.

In [107]:
%%sql

SELECT od.productCode, p.productName, p.productLine, ROUND(od.total_qty_ordered*1.0/p.quantityInStock,2) AS low_stock
  FROM (SELECT productCode, SUM(quantityOrdered) AS total_qty_ordered
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY low_stock DESC
 LIMIT 10;
 

 * sqlite:///stores.db
Done.


productCode,productName,productLine,low_stock
S24_2000,1960 BSA Gold Star DBD34,Motorcycles,67.67
S12_1099,1968 Ford Mustang,Classic Cars,13.72
S32_4289,1928 Ford Phaeton Deluxe,Vintage Cars,7.15
S32_1374,1997 BMW F650 ST,Motorcycles,5.7
S72_3212,Pont Yacht,Ships,2.31
S700_3167,F/A 18 Hornet 1/72,Planes,1.9
S50_4713,2002 Yamaha YZR M1,Motorcycles,1.65
S18_2795,1928 Mercedes-Benz SSK,Vintage Cars,1.61
S18_2248,1911 Ford Town Car,Vintage Cars,1.54
S700_1938,The Mayflower,Ships,1.22


Now lets compute the product performance for each product and get top ten high performance products.

In [106]:
%%sql

SELECT od.productCode, p.productName, p.productLine, od.total_sales
  FROM (SELECT productCode, ROUND(SUM(quantityOrdered * priceEach),2) AS total_sales
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY total_sales DESC
 LIMIT 10;

 * sqlite:///stores.db
Done.


productCode,productName,productLine,total_sales
S18_3232,1992 Ferrari 360 Spider red,Classic Cars,276839.98
S12_1108,2001 Ferrari Enzo,Classic Cars,190755.86
S10_1949,1952 Alpine Renault 1300,Classic Cars,190017.96
S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,170686.0
S12_1099,1968 Ford Mustang,Classic Cars,161531.48
S12_3891,1969 Ford Falcon,Classic Cars,152543.02
S18_1662,1980s Black Hawk Helicopter,Planes,144959.91
S18_2238,1998 Chrysler Plymouth Prowler,Classic Cars,142530.63
S18_1749,1917 Grand Touring Sedan,Vintage Cars,140535.6
S12_2823,2002 Suzuki XREO,Motorcycles,135767.03


Next, let's combine the two queries above using CTEs to get the priority products for restocking.

In [75]:
%%sql

WITH 
low_stock_table AS (
SELECT od.productCode, p.productName, ROUND(od.total_qty_ordered*1.0/p.quantityInStock,2) AS low_stock
  FROM (SELECT productCode, SUM(quantityOrdered) AS total_qty_ordered
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY low_stock DESC
 LIMIT 10
),
performance_table AS(
SELECT od.productCode, p.productName, od.total_sales
  FROM (SELECT productCode, ROUND(SUM(quantityOrdered * priceEach),2) AS total_sales
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY total_sales DESC
 LIMIT 10
)
SELECT productCode, productName, total_sales
  FROM performance_table
 WHERE productCode IN (SELECT productCode
                         FROM low_stock_table);

 * sqlite:///stores.db
Done.


productCode,productName,total_sales
S12_1099,1968 Ford Mustang,161531.48


#### We have found our high performing product that is low on stock. It is 1968 Ford Mustang. This product should be recommeded for ordering more of.

Now, let's also find out some low performing products that are high on stock and we should be ordering less of. Let's run the same queries above but display the results in reverse order.

Get top 10 items which are high on stock.

In [77]:
%%sql
SELECT od.productCode, p.productName, ROUND(od.total_qty_ordered*1.0/p.quantityInStock,2) AS low_stock
  FROM (SELECT productCode, SUM(quantityOrdered) AS total_qty_ordered
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY low_stock 
 LIMIT 10

 * sqlite:///stores.db
Done.


productCode,productName,low_stock
S18_1984,1995 Honda Civic,0.09
S24_3432,2002 Chevy Corvette,0.09
S12_2823,2002 Suzuki XREO,0.1
S12_3380,1968 Dodge Charger,0.1
S18_1589,1965 Aston Martin DB5,0.1
S18_2325,1932 Model A Ford J-Coupe,0.1
S18_2870,1999 Indy 500 Monte Carlo SS,0.1
S18_3482,1976 Ford Gran Torino,0.1
S32_2206,1982 Ducati 996 R,0.1
S700_2466,America West Airlines B757-200,0.1


Next, let's find out 10 lowest performing products.

In [80]:
%%sql
SELECT od.productCode, p.productName, od.total_sales
  FROM (SELECT productCode, ROUND(SUM(quantityOrdered * priceEach),2) AS total_sales
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY total_sales 
 LIMIT 10

 * sqlite:///stores.db
Done.


productCode,productName,total_sales
S24_1937,1939 Chevrolet Deluxe Coupe,28052.94
S24_3969,1936 Mercedes Benz 500k Roadster,29763.39
S24_2972,1982 Lamborghini Diablo,30972.87
S24_2840,1958 Chevy Corvette Limited Edition,31627.96
S32_2206,1982 Ducati 996 R,33268.76
S24_2022,1938 Cadillac V-16 Presidential Limousine,38449.09
S50_1341,1930 Buick Marquette Phaeton,41599.24
S24_1628,1966 Shelby Cobra 427 S/C,42015.54
S72_1253,Boeing X-32A JSF,42692.53
S18_4668,1939 Cadillac Limousine,44037.84


Let's combine the above results to get a low performing product that is high on stock.

In [76]:
%%sql

WITH 
low_stock_table AS (
SELECT od.productCode, p.productName, ROUND(od.total_qty_ordered*1.0/p.quantityInStock,2) AS low_stock
  FROM (SELECT productCode, SUM(quantityOrdered) AS total_qty_ordered
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY low_stock 
 LIMIT 10
),
performance_table AS(
SELECT od.productCode, p.productName, od.total_sales
  FROM (SELECT productCode, ROUND(SUM(quantityOrdered * priceEach),2) AS total_sales
          FROM orderdetails
         GROUP BY productCode) AS od
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY od.productCode, p.productName
 ORDER BY total_sales 
 LIMIT 10
)
SELECT productCode, productName, total_sales
  FROM performance_table
 WHERE productCode IN (SELECT productCode
                         FROM low_stock_table);

 * sqlite:///stores.db
Done.


productCode,productName,total_sales
S32_2206,1982 Ducati 996 R,33268.76


#### We have found our low performing product that is high on stock. It is 1982 Ducati 996 R. This product should be recommeded for orderering less of.

#### Answer 1: Product we should order more of is S12_1099 (1968 Ford Mustang) and product we should order Less of is S32_2206 (1982 Ducati 996 R).	

### Question 2: How Should We Match Marketing and Communication Strategies to Customer Behavior?

This requires categorizing customers: finding the VIP (very important person) customers and those who are less engaged.

- VIP customers bring in the most profit for the store.
- Less-engaged customers bring in less profit.

For example, we could organize some events to drive loyalty for the VIPs and launch a campaign for the less engaged.

In order to categorize customers, let's compute how much profit each customer generates.
Here is the calculation needed to find profit for each customer:  
$profit = SUM(quantityOrdered * (priceEach - buyPrice))$  

We will need these tables:
![](products-orders-orderdetails-tables.png)



In [93]:
%%sql 

-- Create a CTE to get a table where we have sum of quantityOrdered by productCode and by customerNumber
WITH
customer_total AS (
SELECT o.customerNumber, od.productCode, od.priceEach, SUM(od.quantityOrdered) AS total_qty_ordered
  FROM orders o
  JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber, od.productCode, od.priceEach
)

-- Get buyPrice from products table and join it with customer_total table to calculate the profit

SELECT c.customerNumber, ROUND(SUM(c.total_qty_ordered * (c.priceEach - p.buyPrice)),2) AS profit
  FROM customer_total c
  JOIN products p
    ON c.productCode = p.productCode
 GROUP BY c.customerNumber
 ORDER BY profit DESC
 LIMIT 5;

 * sqlite:///stores.db
Done.


customerNumber,profit
141,326519.66
124,236769.39
151,72370.09
114,70311.07
119,60875.3


Now that we have profit by customer table, we also the top five VIP customers (ones with the highest profit).
Let's use the above query as a CTE and then use customers table to get the contact information of top five VIPs - contactLastName, contactFirstName, city, and country


In [98]:
%%sql 

-- Create a CTE to get a table where we have sum of quantityOrdered by productCode and by customerNumber
WITH
customer_total AS (
SELECT o.customerNumber, od.productCode, od.priceEach, SUM(od.quantityOrdered) AS total_qty_ordered
  FROM orders o
  JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber, od.productCode, od.priceEach
),

-- Get buyPrice from products table and join it with customer_total table to calculate the profit
profit_table AS(
SELECT c.customerNumber, ROUND(SUM(c.total_qty_ordered * (c.priceEach - p.buyPrice)),2) AS profit
  FROM customer_total c
  JOIN products p
    ON c.productCode = p.productCode
 GROUP BY c.customerNumber
)

SELECT c.customerNumber, c.contactLastName, c.contactFirstName, c.city, c.country, p.profit
  FROM customers c
  JOIN profit_table p
    ON p.customerNumber = c.customerNumber
 ORDER BY profit DESC
 LIMIT 5;

 * sqlite:///stores.db
Done.


customerNumber,contactLastName,contactFirstName,city,country,profit
141,Freyre,Diego,Madrid,Spain,326519.66
124,Nelson,Susan,San Rafael,USA,236769.39
151,Young,Jeff,NYC,USA,72370.09
114,Ferguson,Peter,Melbourne,Australia,70311.07
119,Labrune,Janine,Nantes,France,60875.3


Next, let's also get the top least-engaged customers. We will reused the above query and just reverse the ordering

In [99]:
%%sql 

-- Create a CTE to get a table where we have sum of quantityOrdered by productCode and by customerNumber
WITH
customer_total AS (
SELECT o.customerNumber, od.productCode, od.priceEach, SUM(od.quantityOrdered) AS total_qty_ordered
  FROM orders o
  JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber, od.productCode, od.priceEach
),

-- Get buyPrice from products table and join it with customer_total table to calculate the profit
profit_table AS(
SELECT c.customerNumber, ROUND(SUM(c.total_qty_ordered * (c.priceEach - p.buyPrice)),2) AS profit
  FROM customer_total c
  JOIN products p
    ON c.productCode = p.productCode
 GROUP BY c.customerNumber
)

SELECT c.customerNumber, c.contactLastName, c.contactFirstName, c.city, c.country, p.profit
  FROM customers c
  JOIN profit_table p
    ON p.customerNumber = c.customerNumber
 ORDER BY profit 
 LIMIT 5;

 * sqlite:///stores.db
Done.


customerNumber,contactLastName,contactFirstName,city,country,profit
219,Young,Mary,Glendale,USA,2610.87
198,Taylor,Leslie,Brickhaven,USA,6586.02
473,Ricotti,Franco,Milan,Italy,9532.93
103,Schmitt,Carine,Nantes,France,10063.8
489,Smith,Thomas,London,UK,10868.04


#### Answer 2: We have found the top five VIPs and top five least engaged customers. We recommend organizing some events to drive loyalty for the VIPs  and launch a campaign for the less engaged.

### Question 3: How Much Can We Spend on Acquiring New Customers?

Before answering this question, let's find the number of new customers arriving each month. That way we can check if it's worth spending money on acquiring new customers.


In [102]:
%%sql

WITH 

payment_with_year_month_table AS (
SELECT *, 
       CAST(SUBSTR(paymentDate, 1,4) AS INTEGER)*100 + CAST(SUBSTR(paymentDate, 6,7) AS INTEGER) AS year_month
  FROM payments p
),

customers_by_month_table AS (
SELECT p1.year_month, COUNT(*) AS number_of_customers, SUM(p1.amount) AS total
  FROM payment_with_year_month_table p1
 GROUP BY p1.year_month
),

new_customers_by_month_table AS (
SELECT p1.year_month, 
       COUNT(DISTINCT customerNumber) AS number_of_new_customers,
       SUM(p1.amount) AS new_customer_total,
       (SELECT number_of_customers
          FROM customers_by_month_table c
        WHERE c.year_month = p1.year_month) AS number_of_customers,
       (SELECT total
          FROM customers_by_month_table c
         WHERE c.year_month = p1.year_month) AS total
  FROM payment_with_year_month_table p1
 WHERE p1.customerNumber NOT IN (SELECT customerNumber
                                   FROM payment_with_year_month_table p2
                                  WHERE p2.year_month < p1.year_month)
 GROUP BY p1.year_month
)

SELECT year_month, 
       ROUND(number_of_new_customers*100/number_of_customers,1) AS number_of_new_customers_props,
       ROUND(new_customer_total*100/total,1) AS new_customers_total_props
  FROM new_customers_by_month_table;

 * sqlite:///stores.db
Done.


year_month,number_of_new_customers_props,new_customers_total_props
200301,100.0,100.0
200302,100.0,100.0
200303,100.0,100.0
200304,100.0,100.0
200305,83.0,100.0
200306,100.0,100.0
200307,75.0,68.3
200308,66.0,54.2
200309,80.0,95.9
200310,69.0,69.3


The number of clients has been decreasing since 2003, and in 2004, we had the lowest values. The year 2005, which is present in the database as well, isn't present in the table above, this means that the store has not had any new customers since September of 2004. This means it makes sense to spend money acquiring new customers.

To determine how much money we can spend acquiring new customers, we can compute the Customer Lifetime Value (LTV), which represents the average amount of money a customer generates. We can then determine how much we can spend on marketing.

Now, let's compute the average of customer profits using the CTE we used above.

In [103]:
%%sql 

-- Create a CTE to get a table where we have sum of quantityOrdered by productCode and by customerNumber
WITH
customer_total AS (
SELECT o.customerNumber, od.productCode, od.priceEach, SUM(od.quantityOrdered) AS total_qty_ordered
  FROM orders o
  JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber, od.productCode, od.priceEach
),

-- Get buyPrice from products table and join it with customer_total table to calculate the profit
profit_table AS(
SELECT c.customerNumber, ROUND(SUM(c.total_qty_ordered * (c.priceEach - p.buyPrice)),2) AS profit
  FROM customer_total c
  JOIN products p
    ON c.productCode = p.productCode
 GROUP BY c.customerNumber
)

SELECT AVG(profit)
  FROM profit_table;

 * sqlite:///stores.db
Done.


AVG(profit)
39039.5943877551


## Conclusion
Question 1: Which products should we order more of or less of?  
Answer 1: Product we should order more of is S12_1099 (1968 Ford Mustang) and product we should order Less of is S32_2206 (1982 Ducati 996 R).

Question 2: How should we match marketing and communication strategies to customer behaviors?  
Answer 2: We have the most-important and least-committed customers, we can determine how to drive loyalty and attract more customers.

Question 3: How much can we spend on acquiring new customers?  
Answer 3: LTV tells us how much profit an average customer generates during their lifetime with our store. We can use it to predict our future profit. So, if we get ten new customers next month, we'll earn 390,395 dollars, and we can decide based on this prediction how much we can spend on acquiring new customers.




## Author
Puneet Pawar